# Demonstration of WellResponse Class

Notebook to demonstrate using the Well Class to 
estimate drawdown or stream depletion using the 
solutions available in the python module.

In [ ]:
import sys
sys.path.insert(1, '../../')
import pycap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Example Depletion
The Well class will estimate depletions for streams.  The stream distances and
streambed conductances, if needed, are passed through dictionaries keyed by 
a stream name or ID.  In this notebook a table of values will be made manually from 
Table 2 from Reeves, H.W., Hamilton, D.A., Seelbach, P.W., and Asher, A.J., 2009, Ground-water-withdrawal component of the Michigan water-withdrawal screening tool: U.S. Geological Survey Scientific Investigations Report 2009–5003, 36 p.
[https://pubs.usgs.gov/sir/2009/5003/]

In [ ]:
stream_table = pd.DataFrame(({'id': 8, 'distance': 14802},
                            {'id': 9, 'distance': 12609.2},
                            {'id': 11, 'distance': 15750.5},
                            {'id': 27, 'distance': 22567.6},
                            {'id': 9741, 'distance': 27565.2},
                            {'id': 10532, 'distance': 33059.5},
                            {'id': 11967, 'distance': 14846.3},
                            {'id': 12515, 'distance': 17042.55},
                            {'id': 12573, 'distance': 11959.5},
                            {'id': 12941, 'distance': 19070.8},
                            {'id': 13925, 'distance': 10028.9}))



In [ ]:
stream_table.head()

# Compute apportionment

The Michigan screening tool uses inverse distance weighting apportionment.  Other
apportionment approaches that may be used as a simple way to extend the analytical
solution are discussed by Zipper and others (2019).  [https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2018WR024403]

In [ ]:
invers =np.array([1/x for x in stream_table['distance']])
stream_table['apportionment'] = (1./stream_table['distance'])/np.sum(invers)

In [ ]:
stream_table

In [ ]:
np.sum(stream_table['apportionment'])

make dictionaries from the distances and apportionment values

In [ ]:
distances = dict(zip(stream_table.id.values, stream_table.distance.values))

In [ ]:
apportion = dict(zip(stream_table.id.values, stream_table.apportionment.values))

# set aquifer properties and streambed conductances

In [ ]:
stream_table = pd.DataFrame(({'id': 8, 'distance': 14802},
                            {'id': 9, 'distance': 12609.2},
                            {'id': 11, 'distance': 15750.5},
                            {'id': 27, 'distance': 22567.6},
                            {'id': 9741, 'distance': 27565.2},
                            {'id': 10532, 'distance': 33059.5},
                            {'id': 11967, 'distance': 14846.3},
                            {'id': 12515, 'distance': 17042.5},
                            {'id': 12573, 'distance': 11959.5},
                            {'id': 12941, 'distance': 19070.8},
                            {'id': 13925, 'distance': 10028.9}))

In [ ]:
T= 7211.  # ft^2/day
S= 0.01
Q = 70  # 70 gpm in cubic feet per day
well_name = 'demo'
pumpdays = int(5. * 365)

## call a utility to convert Q to a series with appropriate formatting

also convert to CFD from GPM

In [ ]:
Q = pycap.Q2ts(pumpdays, 5, Q) * pycap.GPM2CFD

In [ ]:
Q

In [ ]:
stream_table['conductance'] = 7.11855  # example uses constant streambed_conductance

In [ ]:
stream_table.head()

In [ ]:
cond = dict(zip(stream_table.id.values, stream_table.conductance.values))

In [ ]:
test_well = pycap.Well(T=T,
                 S=S,
                 Q=Q,
                 depletion_years=5,
                 depl_method='hunt99',
                 drawdown_dist={'test0':50.},
                 streambed_conductance=cond,
                 stream_dist=distances,
                 stream_apportionment=apportion)

In [ ]:
test_well.drawdown

In [ ]:
deltaQ = pycap._calc_deltaQ(Q)
deltaQ

In [ ]:
test_well.depletion

In [ ]:
stream_depl = pd.DataFrame(test_well.depletion)

## need to convert to GPM to compare results to Table 2

In [ ]:
stream_depl = stream_depl/pycap.GPM2CFD
stream_depl

In [ ]:
five_year= pd.DataFrame(stream_depl.loc[1824].T)

In [ ]:
five_year.rename(columns={1824: 'Depletion'}, inplace=True)

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
five_year

# Matches Table 2